# AI-Driven Early Prediction of Pulmonary Fibrosis Using Deep Learning


In [10]:
import os
import pandas as pd
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings

warnings.filterwarnings('ignore')

# ==========================================
# 1. CONFIGURATION
# ==========================================
CONFIG = {
    "lr": 2e-3,
    "weight_decay": 1e-4,
    "batch_size": 128,
    "epochs": 200,
    "n_folds": 5,
    "quantiles": [0.2, 0.5, 0.8], 
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    "data_dir": "../input/osic-pulmonary-fibrosis-progression",
    # Point to your extracted features file
    "biomarker_path": "../input/feature-extraction-u-net-segmentation/master_dataset.csv"
}

# ==========================================
# 2. DATA PREPROCESSING (ROBUST MERGE)
# ==========================================
def preprocess_data(config):
    global TARGET_SCALER
    
    # 1. Load Clinical Data (Guarantees Sex, Age, Weeks exist)
    clinical_df = pd.read_csv(f"{config['data_dir']}/train.csv")
    
    # 2. Load Biomarkers
    biomarkers_df = pd.read_csv(config['biomarker_path'])
    
    # 3. Robust Merge Logic
    # We only want the image features from the biomarkers file to avoid duplicates
    image_features = [
        'lung_vol_ml', 'hu_mean', 'hu_std', 'hu_skew', 'hu_kurt',
        'glcm_contrast', 'glcm_homogeneity', 'glcm_energy', 'glcm_correlation'
    ]
    
    # Filter biomarkers_df to only keep Patient + Image Features
    # This prevents 'Sex_x', 'Sex_y' conflicts if columns repeat
    cols_to_keep = ['Patient'] + [c for c in image_features if c in biomarkers_df.columns]
    
    # Remove duplicates in biomarkers (keep 1 row per patient)
    biomarkers_clean = biomarkers_df[cols_to_keep].drop_duplicates(subset=['Patient'])
    
    # MERGE: Clinical (Left) + Biomarkers (Right)
    train = clinical_df.merge(biomarkers_clean, on='Patient', how='inner')
    
    # 4. Feature Engineering (Baseline & Relative Weeks)
    train['Weeks'] = train['Weeks'].astype(int)
    train.sort_values(['Patient', 'Weeks'], inplace=True)
    
    baseline = train.groupby('Patient').first().reset_index()
    baseline = baseline[['Patient', 'FVC', 'Percent']].rename(columns={'FVC': 'Base_FVC', 'Percent': 'Base_Percent'})
    train = train.merge(baseline, on='Patient', how='left')
    
    base_weeks = train.groupby('Patient')['Weeks'].min().reset_index().rename(columns={'Weeks': 'Base_Week'})
    train = train.merge(base_weeks, on='Patient', how='left')
    train['Relative_Weeks'] = train['Weeks'] - train['Base_Week']
    
    # 5. Feature Scaling
    scaler = RobustScaler()
    # Dynamic check for available columns
    available_img_feats = [c for c in image_features if c in train.columns]
    
    num_cols = ['Age', 'Base_FVC', 'Base_Percent', 'Relative_Weeks'] + available_img_feats
    train[num_cols] = scaler.fit_transform(train[num_cols])
    
    # 6. Encoding (Now guaranteed to work)
    train['Sex'] = train['Sex'].apply(lambda x: 1 if x == 'Male' else 0)
    train['Smk_Ex'] = train['SmokingStatus'].apply(lambda x: 1 if x == 'Ex-smoker' else 0)
    train['Smk_Cur'] = train['SmokingStatus'].apply(lambda x: 1 if x == 'Currently smokes' else 0)
    
    feature_cols = num_cols + ['Sex', 'Smk_Ex', 'Smk_Cur']
    
    # 7. Target Scaling
    TARGET_SCALER.fit(train[['FVC']])
    train['FVC_scaled'] = TARGET_SCALER.transform(train[['FVC']])
    
    print(f"✅ Preprocessing Complete. Final Shape: {train.shape}")
    print(f"✅ Features Used: {len(feature_cols)}")
    
    return train, feature_cols

# ==========================================
# 3. MODEL: QUANTILE MLP
# ==========================================
class QuantileMLP(nn.Module):
    def __init__(self, input_dim, quantiles):
        super().__init__()
        hidden = 128
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden), nn.BatchNorm1d(hidden), nn.LeakyReLU(0.1), nn.Dropout(0.3),
            nn.Linear(hidden, hidden), nn.BatchNorm1d(hidden), nn.LeakyReLU(0.1), nn.Dropout(0.3),
            nn.Linear(hidden, len(quantiles))
        )
    def forward(self, x): return self.net(x)

def quantile_loss(preds, target, quantiles):
    """Pinball Loss on SCALED targets."""
    assert not target.requires_grad
    losses = []
    for i, q in enumerate(quantiles):
        errors = target - preds[:, i].unsqueeze(1)
        loss = torch.max((q-1) * errors, q * errors)
        losses.append(loss)
    return torch.mean(torch.sum(torch.cat(losses, dim=1), dim=1))

# ==========================================
# 4. METRIC CALCULATION
# ==========================================
def calculate_metrics(y_true, q_preds_real):
    """
    Calculates metrics on UNSCALED (Real) values.
    """
    q20 = q_preds_real[:, 0]
    q50 = q_preds_real[:, 1]
    q80 = q_preds_real[:, 2]
    
    # Sigma (Uncertainty)
    sigma = q80 - q20
    sigma_clipped = np.maximum(sigma, 70)
    
    # LLL
    delta = np.minimum(np.abs(y_true - q50), 1000)
    lll = - (np.sqrt(2) * delta / sigma_clipped) - np.log(np.sqrt(2) * sigma_clipped)
    
    # Regression Metrics
    mse = mean_squared_error(y_true, q50)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, q50)
    r2 = r2_score(y_true, q50)
    rmae = mae / (np.mean(np.abs(y_true)) + 1e-6)
    
    return np.mean(lll), mse, rmse, mae, rmae, r2

# ==========================================
# 5. TRAINING LOOP
# ==========================================
def train_model():
    df, features = preprocess_data(CONFIG)
    patients = df['Patient'].unique()
    kf = KFold(n_splits=CONFIG['n_folds'], shuffle=True, random_state=42)
    
    global_trues, global_preds = [], []
    
    print(f"🚀 Training Direct FVC Quantile Regression on {len(df)} visits.")
    print(f"Features: {features}")
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(patients)):
        train_p, val_p = patients[train_idx], patients[val_idx]
        train_data = df[df['Patient'].isin(train_p)]
        val_data = df[df['Patient'].isin(val_p)]
        
        # Train on SCALED FVC
        X_train = torch.tensor(train_data[features].values, dtype=torch.float32).to(CONFIG['device'])
        y_train_scaled = torch.tensor(train_data['FVC_scaled'].values, dtype=torch.float32).unsqueeze(1).to(CONFIG['device'])
        
        X_val = torch.tensor(val_data[features].values, dtype=torch.float32).to(CONFIG['device'])
        
        # Validation on RAW FVC (for final metrics)
        y_val_real = val_data['FVC'].values
        
        model = QuantileMLP(len(features), CONFIG['quantiles']).to(CONFIG['device'])
        optimizer = optim.AdamW(model.parameters(), lr=CONFIG['lr'], weight_decay=CONFIG['weight_decay'])
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=CONFIG['epochs'], eta_min=1e-6)
        
        best_lll = -float('inf')
        best_preds_real = None
        
        for epoch in range(CONFIG['epochs']):
            model.train()
            optimizer.zero_grad()
            preds_scaled = model(X_train)
            
            # Loss on SCALED targets
            loss = quantile_loss(preds_scaled, y_train_scaled, CONFIG['quantiles'])
            loss.backward()
            optimizer.step()
            scheduler.step()
            
            # Validation
            model.eval()
            with torch.no_grad():
                val_preds_scaled = model(X_val)
                
                # UNSCALE Predictions for metrics
                val_preds_real = TARGET_SCALER.inverse_transform(val_preds_scaled.cpu().numpy())
                
                lll, _, _, _, _, _ = calculate_metrics(y_val_real, val_preds_real)
                
            if lll > best_lll:
                best_lll = lll
                best_preds_real = val_preds_real
                
        print(f"Fold {fold+1} Best | LLL: {best_lll:.4f}")
        
        global_trues.extend(y_val_real)
        global_preds.extend(best_preds_real)

    # --- FINAL SCORE ---
    g_true = np.array(global_trues)
    g_pred = np.array(global_preds) 
    
    lll, mse, rmse, mae, rmae, r2 = calculate_metrics(g_true, g_pred)
    
    print("\n" + "="*50)
    print("🏆 FINAL DIRECT FVC QUANTILE RESULTS")
    print("="*50)
    print(f"R² (FVC Correlation) : {r2:.4f}   (Target > 0.88)")
    print(f"MSE (mL²)            : {mse:.4f}")
    print(f"RMSE (mL)            : {rmse:.4f}  (Target < 170)")
    print(f"MAE (mL)             : {mae:.4f}")
    print(f"RMAE (Relative Error): {rmae:.4f}")
    print(f"LLL (OSIC Metric)    : {lll:.4f}   (Target > -6.64)")
    print("="*50)

if __name__ == "__main__":
    train_model()

✅ Preprocessing Complete. Final Shape: (1549, 23)
✅ Features Used: 16
🚀 Training Direct FVC Quantile Regression on 1549 visits.
Features: ['Age', 'Base_FVC', 'Base_Percent', 'Relative_Weeks', 'lung_vol_ml', 'hu_mean', 'hu_std', 'hu_skew', 'hu_kurt', 'glcm_contrast', 'glcm_homogeneity', 'glcm_energy', 'glcm_correlation', 'Sex', 'Smk_Ex', 'Smk_Cur']
Fold 1 Best | LLL: -6.8668
Fold 2 Best | LLL: -7.0244
Fold 3 Best | LLL: -6.9294
Fold 4 Best | LLL: -6.8974
Fold 5 Best | LLL: -6.8904

🏆 FINAL DIRECT FVC QUANTILE RESULTS
R² (FVC Correlation) : 0.9014   (Target > 0.88)
MSE (mL²)            : 68311.0493
RMSE (mL)            : 261.3638  (Target < 170)
MAE (mL)             : 184.9357
RMAE (Relative Error): 0.0687
LLL (OSIC Metric)    : -6.9214   (Target > -6.64)


## Hurray

In [ ]:
✅ Preprocessing Complete. Final Shape: (1549, 23)
✅ Features Used: 16
🚀 Training Direct FVC Quantile Regression on 1549 visits.
Features: ['Age', 'Base_FVC', 'Base_Percent', 'Relative_Weeks', 'lung_vol_ml', 'hu_mean', 'hu_std', 'hu_skew', 'hu_kurt', 'glcm_contrast', 'glcm_homogeneity', 'glcm_energy', 'glcm_correlation', 'Sex', 'Smk_Ex', 'Smk_Cur']
Fold 1 Best | LLL: -6.8408
Fold 2 Best | LLL: -6.9963
Fold 3 Best | LLL: -6.8772
Fold 4 Best | LLL: -6.9388
Fold 5 Best | LLL: -6.8931

==================================================
🏆 FINAL DIRECT FVC QUANTILE RESULTS
==================================================
R² (FVC Correlation) : 0.9049   (Target > 0.88)
MSE (mL²)            : 65878.0976
RMSE (mL)            : 256.6673  (Target < 170)
MAE (mL)             : 185.8179
RMAE (Relative Error): 0.0691
LLL (OSIC Metric)    : -6.9088   (Target > -6.64)
==================================================

# Phase 2 - Fine Tunining

In [11]:
import os
import pandas as pd
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings

warnings.filterwarnings('ignore')

# ==========================================
# 1. ENHANCED CONFIGURATION
# ==========================================
CONFIG = {
    "lr": 1.5e-3,  # Slightly reduced for stability
    "weight_decay": 5e-5,  # Reduced regularization
    "batch_size": 64,  # Smaller batches for better generalization
    "epochs": 300,  # More epochs with patience
    "n_folds": 5,
    "quantiles": [0.2, 0.5, 0.8], 
    "patience": 50,  # Early stopping
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    "data_dir": "../input/osic-pulmonary-fibrosis-progression",
    "biomarker_path": "../input/feature-extraction-u-net-segmentation/master_dataset.csv",
    "seed": 42
}

# Set seeds for reproducibility
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(CONFIG['seed'])

# ==========================================
# 2. ENHANCED DATA PREPROCESSING
# ==========================================
def preprocess_data(config):
    # 1. Load Clinical Data
    clinical_df = pd.read_csv(f"{config['data_dir']}/train.csv")
    
    # 2. Load Biomarkers
    biomarkers_df = pd.read_csv(config['biomarker_path'])
    
    # 3. Robust Merge Logic
    image_features = [
        'lung_vol_ml', 'hu_mean', 'hu_std', 'hu_skew', 'hu_kurt',
        'glcm_contrast', 'glcm_homogeneity', 'glcm_energy', 'glcm_correlation'
    ]
    
    cols_to_keep = ['Patient'] + [c for c in image_features if c in biomarkers_df.columns]
    biomarkers_clean = biomarkers_df[cols_to_keep].drop_duplicates(subset=['Patient'])
    
    train = clinical_df.merge(biomarkers_clean, on='Patient', how='inner')
    
    # 4. Enhanced Feature Engineering
    train['Weeks'] = train['Weeks'].astype(int)
    train.sort_values(['Patient', 'Weeks'], inplace=True)
    
    # Baseline features
    baseline = train.groupby('Patient').first().reset_index()
    baseline = baseline[['Patient', 'FVC', 'Percent']].rename(
        columns={'FVC': 'Base_FVC', 'Percent': 'Base_Percent'}
    )
    train = train.merge(baseline, on='Patient', how='left')
    
    base_weeks = train.groupby('Patient')['Weeks'].min().reset_index().rename(
        columns={'Weeks': 'Base_Week'}
    )
    train = train.merge(base_weeks, on='Patient', how='left')
    train['Relative_Weeks'] = train['Weeks'] - train['Base_Week']
    
    # NEW: Add interaction features (critical for reducing RMSE)
    train['FVC_Week_Interaction'] = train['Base_FVC'] * train['Relative_Weeks']
    train['Age_Week_Interaction'] = train['Age'] * train['Relative_Weeks']
    
    available_img_feats = [c for c in image_features if c in train.columns]
    
    # NEW: Add lung volume decline rate proxy
    if 'lung_vol_ml' in train.columns:
        train['LungVol_FVC_Ratio'] = train['lung_vol_ml'] / (train['Base_FVC'] + 1e-6)
    
    # 5. Feature Scaling (Using StandardScaler for better tail behavior)
    scaler = StandardScaler()
    
    num_cols = ['Age', 'Base_FVC', 'Base_Percent', 'Relative_Weeks'] + available_img_feats
    interaction_cols = ['FVC_Week_Interaction', 'Age_Week_Interaction']
    
    if 'LungVol_FVC_Ratio' in train.columns:
        interaction_cols.append('LungVol_FVC_Ratio')
    
    all_num_cols = num_cols + interaction_cols
    train[all_num_cols] = scaler.fit_transform(train[all_num_cols])
    
    # 6. Encoding
    train['Sex'] = train['Sex'].apply(lambda x: 1 if x == 'Male' else 0)
    train['Smk_Ex'] = train['SmokingStatus'].apply(lambda x: 1 if x == 'Ex-smoker' else 0)
    train['Smk_Cur'] = train['SmokingStatus'].apply(lambda x: 1 if x == 'Currently smokes' else 0)
    
    feature_cols = all_num_cols + ['Sex', 'Smk_Ex', 'Smk_Cur']
    
    # 7. Target Scaling (Use StandardScaler for FVC)
    target_scaler = StandardScaler()
    target_scaler.fit(train[['FVC']])
    train['FVC_scaled'] = target_scaler.transform(train[['FVC']])
    
    print(f"✅ Preprocessing Complete. Final Shape: {train.shape}")
    print(f"✅ Features Used: {len(feature_cols)}")
    
    return train, feature_cols, target_scaler

# ==========================================
# 3. IMPROVED MODEL: DEEPER QUANTILE MLP
# ==========================================
class EnhancedQuantileMLP(nn.Module):
    def __init__(self, input_dim, quantiles, dropout=0.25):
        super().__init__()
        # Wider and deeper architecture
        h1, h2, h3 = 256, 128, 64
        
        self.net = nn.Sequential(
            nn.Linear(input_dim, h1),
            nn.BatchNorm1d(h1),
            nn.LeakyReLU(0.1),
            nn.Dropout(dropout),
            
            nn.Linear(h1, h2),
            nn.BatchNorm1d(h2),
            nn.LeakyReLU(0.1),
            nn.Dropout(dropout),
            
            nn.Linear(h2, h3),
            nn.BatchNorm1d(h3),
            nn.LeakyReLU(0.1),
            nn.Dropout(dropout),
            
            nn.Linear(h3, len(quantiles))
        )
    
    def forward(self, x):
        return self.net(x)

def quantile_loss(preds, target, quantiles):
    """Pinball Loss on SCALED targets."""
    assert not target.requires_grad
    losses = []
    for i, q in enumerate(quantiles):
        errors = target - preds[:, i].unsqueeze(1)
        loss = torch.max((q-1) * errors, q * errors)
        losses.append(loss)
    return torch.mean(torch.sum(torch.cat(losses, dim=1), dim=1))

# ==========================================
# 4. METRIC CALCULATION (UNCHANGED)
# ==========================================
def calculate_metrics(y_true, q_preds_real):
    q20 = q_preds_real[:, 0]
    q50 = q_preds_real[:, 1]
    q80 = q_preds_real[:, 2]
    
    sigma = q80 - q20
    sigma_clipped = np.maximum(sigma, 70)
    
    delta = np.minimum(np.abs(y_true - q50), 1000)
    lll = - (np.sqrt(2) * delta / sigma_clipped) - np.log(np.sqrt(2) * sigma_clipped)
    
    mse = mean_squared_error(y_true, q50)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, q50)
    r2 = r2_score(y_true, q50)
    rmae = mae / (np.mean(np.abs(y_true)) + 1e-6)
    
    return np.mean(lll), mse, rmse, mae, rmae, r2

# ==========================================
# 5. ENHANCED TRAINING LOOP
# ==========================================
def train_model():
    df, features, target_scaler = preprocess_data(CONFIG)
    patients = df['Patient'].unique()
    kf = KFold(n_splits=CONFIG['n_folds'], shuffle=True, random_state=CONFIG['seed'])
    
    global_trues, global_preds = [], []
    fold_models = []
    
    print(f"🚀 Training Enhanced Quantile Regression on {len(df)} visits.")
    print(f"Features ({len(features)}): {features[:5]}... (showing first 5)")
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(patients)):
        train_p, val_p = patients[train_idx], patients[val_idx]
        train_data = df[df['Patient'].isin(train_p)]
        val_data = df[df['Patient'].isin(val_p)]
        
        X_train = torch.tensor(train_data[features].values, dtype=torch.float32).to(CONFIG['device'])
        y_train_scaled = torch.tensor(train_data['FVC_scaled'].values, dtype=torch.float32).unsqueeze(1).to(CONFIG['device'])
        
        X_val = torch.tensor(val_data[features].values, dtype=torch.float32).to(CONFIG['device'])
        y_val_real = val_data['FVC'].values
        
        model = EnhancedQuantileMLP(len(features), CONFIG['quantiles']).to(CONFIG['device'])
        optimizer = optim.AdamW(model.parameters(), lr=CONFIG['lr'], weight_decay=CONFIG['weight_decay'])
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='max', factor=0.5, patience=20, verbose=False
        )
        
        best_lll = -float('inf')
        best_preds_real = None
        patience_counter = 0
        
        for epoch in range(CONFIG['epochs']):
            model.train()
            optimizer.zero_grad()
            preds_scaled = model(X_train)
            
            loss = quantile_loss(preds_scaled, y_train_scaled, CONFIG['quantiles'])
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping
            optimizer.step()
            
            # Validation
            model.eval()
            with torch.no_grad():
                val_preds_scaled = model(X_val)
                val_preds_real = target_scaler.inverse_transform(val_preds_scaled.cpu().numpy())
                
                lll, _, rmse, _, _, _ = calculate_metrics(y_val_real, val_preds_real)
                
            scheduler.step(lll)
            
            if lll > best_lll:
                best_lll = lll
                best_preds_real = val_preds_real
                patience_counter = 0
            else:
                patience_counter += 1
                
            if patience_counter >= CONFIG['patience']:
                print(f"  Early stopping at epoch {epoch+1}")
                break
                
        print(f"Fold {fold+1} Best | LLL: {best_lll:.4f}")
        
        global_trues.extend(y_val_real)
        global_preds.extend(best_preds_real)
        fold_models.append(model)

    # --- FINAL SCORE ---
    g_true = np.array(global_trues)
    g_pred = np.array(global_preds) 
    
    lll, mse, rmse, mae, rmae, r2 = calculate_metrics(g_true, g_pred)
    
    print("\n" + "="*60)
    print("🏆 FINAL ENHANCED QUANTILE RESULTS")
    print("="*60)
    print(f"R² (FVC Correlation) : {r2:.4f}   (Target > 0.88) {'✅' if r2 > 0.88 else '❌'}")
    print(f"MSE (mL²)            : {mse:.4f}")
    print(f"RMSE (mL)            : {rmse:.4f}  (Target < 170) {'✅' if rmse < 170 else '❌'}")
    print(f"MAE (mL)             : {mae:.4f}")
    print(f"RMAE (Relative Error): {rmae:.4f}")
    print(f"LLL (OSIC Metric)    : {lll:.4f}   (Target > -6.64) {'✅' if lll > -6.64 else '❌'}")
    print("="*60)
    
    return fold_models, target_scaler

if __name__ == "__main__":
    models, scaler = train_model()

✅ Preprocessing Complete. Final Shape: (1549, 26)
✅ Features Used: 19
🚀 Training Enhanced Quantile Regression on 1549 visits.
Features (19): ['Age', 'Base_FVC', 'Base_Percent', 'Relative_Weeks', 'lung_vol_ml']... (showing first 5)
  Early stopping at epoch 165
Fold 1 Best | LLL: -6.8198
  Early stopping at epoch 103
Fold 2 Best | LLL: -6.9550
  Early stopping at epoch 164
Fold 3 Best | LLL: -6.8752
  Early stopping at epoch 101
Fold 4 Best | LLL: -6.9504
  Early stopping at epoch 96
Fold 5 Best | LLL: -6.8886

🏆 FINAL ENHANCED QUANTILE RESULTS
R² (FVC Correlation) : 0.9132   (Target > 0.88) ✅
MSE (mL²)            : 60135.3067
RMSE (mL)            : 245.2250  (Target < 170) ❌
MAE (mL)             : 176.6318
RMAE (Relative Error): 0.0657
LLL (OSIC Metric)    : -6.8973   (Target > -6.64) ❌


### 1. The Diagnosis: The "Floating Curve" Syndrome
Your model has learned the disease mechanism perfectly, but it has failed to learn the calibration.High $R^2$ (0.91): Your model knows exactly who is sick and who is getting worse. If Patient A drops 50mL and Patient B drops 200mL, your model correctly ranks them. The U-Net features are working.High RMSE (245 vs 170): The model is "floating." It predicts the correct shape of the decline, but the whole curve is shifted up or down by a constant amount (bias). It knows the slope, but it's missing the intercept.The Cause: You are feeding Baseline FVC as just another feature (like Age or Sex) into a neural network. Neural networks introduce noise. The model sees "Baseline = 2500" but outputs "Prediction = 2650" even for Week 0, because it's approximating, not calculating.The Fix: You need to force the model to respect the Anchor

###  Point.2. Strategy A: Post-Processing "Anchoring" (The Quickest Fix)
This requires zero retraining. It is a mathematical adjustment applied to your final predictions.The Logic:We know for a fact that at Week 0 (Baseline), the error must be zero. Currently, your model likely has an error at Week 0.If your model predicts 2400 at Week 0, but the real value is 2500, your model has a -100 bias for that patient. It is highly probable that this -100 bias persists for Week 10, Week 50, etc.The Solution:Calculate the error your model makes at the Baseline week for each patient. Add that error back to every future prediction for that same patient. This "shifts" the floating curve to snap onto the known starting point. In similar competitions, this single step often drops RMSE by 30-50

### points.3. Strategy B: Uncertainty "Clipping" (Fixing LLL)Your LLL is -6.90 (Target -6.64). 
LLL is damaged by two things: being wrong (RMSE) and being "overconfidently wrong."The Logic:Your model is outputting a confidence interval ($\sigma$). Currently, it is likely predicting very large sigmas to protect itself from the high RMSE.However, once you apply the "Anchoring" fix (Strategy A), your RMSE will drop. Your model doesn't know this yet, so it will still output huge, pessimistic uncertainty values.The Solution:After improving RMSE via anchoring, you must artificially tighten the confidence intervals. You can multiply your predicted $\sigma$ (uncertainty) by a factor (e.g., $0.8$ or $0.9$). Since your predictions are now more accurate, you should claim more confidence. This will boost the LLL score directly.
### 4. Strategy C: The "Decay Factor" Approach (Architectural Shift)
If the above post-processing doesn't fully close the gap, the issue is how the problem is framed for the Neural Network.The Logic:Currently, you ask the model: "Here is a lung and an age. Guess the FVC in mL." This is hard because FVC ranges from 1,000 to 6,000.It is much easier to ask: "Here is a lung and an age. Predict the percentage of the Baseline FVC remaining."The Solution:Change the target. Instead of predicting raw FVC, train the model to predict Ratio = FVC_current / FVC_baseline.This normalizes every patient to start at 1.0.The model only has to learn the decay rate (e.g., 0.98, 0.95).At inference time, you simply multiply Predicted_Ratio * Real_Baseline_FVC.This forces the model to heavily rely on the known baseline, mechanically reducing variance.

### Summary of the Plan : You do not need more complex code. 
You need Calibration.Do not retrain yet. Take your existing OOF predictions from the last run.Apply Anchoring: For every patient, calculate Shift = Real_Baseline - Predicted_Baseline. Add this Shift to all predictions. Check the new RMSE.Calibrate Sigma: If RMSE improves, multiply your uncertainty column by a scalar (try 0.9, 0.8) to maximize LLL.If this brings RMSE close to 200 but not 170, then we switch to Strategy C (Ratio Prediction) for the final model.

In [14]:
import os
import pandas as pd
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings

warnings.filterwarnings('ignore')

# ==========================================
# 1. CONFIGURATION
# ==========================================
CONFIG = {
    "lr": 1.5e-3,
    "weight_decay": 5e-5,
    "batch_size": 64,
    "epochs": 300,
    "n_folds": 5,
    "quantiles": [0.2, 0.5, 0.8], 
    "patience": 50,
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    "data_dir": "../input/osic-pulmonary-fibrosis-progression",
    "biomarker_path": "../input/feature-extraction-u-net-segmentation/master_dataset.csv",
    "seed": 42
}

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(CONFIG['seed'])

# ==========================================
# 2. DATA PREPROCESSING
# ==========================================
def preprocess_data(config):
    clinical_df = pd.read_csv(f"{config['data_dir']}/train.csv")
    biomarkers_df = pd.read_csv(config['biomarker_path'])
    
    image_features = [
        'lung_vol_ml', 'hu_mean', 'hu_std', 'hu_skew', 'hu_kurt',
        'glcm_contrast', 'glcm_homogeneity', 'glcm_energy', 'glcm_correlation'
    ]
    
    cols_to_keep = ['Patient'] + [c for c in image_features if c in biomarkers_df.columns]
    biomarkers_clean = biomarkers_df[cols_to_keep].drop_duplicates(subset=['Patient'])
    
    train = clinical_df.merge(biomarkers_clean, on='Patient', how='inner')
    
    train['Weeks'] = train['Weeks'].astype(int)
    train.sort_values(['Patient', 'Weeks'], inplace=True)
    
    baseline = train.groupby('Patient').first().reset_index()
    baseline = baseline[['Patient', 'FVC', 'Percent']].rename(
        columns={'FVC': 'Base_FVC', 'Percent': 'Base_Percent'}
    )
    train = train.merge(baseline, on='Patient', how='left')
    
    base_weeks = train.groupby('Patient')['Weeks'].min().reset_index().rename(
        columns={'Weeks': 'Base_Week'}
    )
    train = train.merge(base_weeks, on='Patient', how='left')
    train['Relative_Weeks'] = train['Weeks'] - train['Base_Week']
    
    # Interaction features
    train['FVC_Week_Interaction'] = train['Base_FVC'] * train['Relative_Weeks']
    train['Age_Week_Interaction'] = train['Age'] * train['Relative_Weeks']
    
    available_img_feats = [c for c in image_features if c in train.columns]
    
    if 'lung_vol_ml' in train.columns:
        train['LungVol_FVC_Ratio'] = train['lung_vol_ml'] / (train['Base_FVC'] + 1e-6)
    
    scaler = StandardScaler()
    
    num_cols = ['Age', 'Base_FVC', 'Base_Percent', 'Relative_Weeks'] + available_img_feats
    interaction_cols = ['FVC_Week_Interaction', 'Age_Week_Interaction']
    
    if 'LungVol_FVC_Ratio' in train.columns:
        interaction_cols.append('LungVol_FVC_Ratio')
    
    all_num_cols = num_cols + interaction_cols
    train[all_num_cols] = scaler.fit_transform(train[all_num_cols])
    
    train['Sex'] = train['Sex'].apply(lambda x: 1 if x == 'Male' else 0)
    train['Smk_Ex'] = train['SmokingStatus'].apply(lambda x: 1 if x == 'Ex-smoker' else 0)
    train['Smk_Cur'] = train['SmokingStatus'].apply(lambda x: 1 if x == 'Currently smokes' else 0)
    
    feature_cols = all_num_cols + ['Sex', 'Smk_Ex', 'Smk_Cur']
    
    target_scaler = StandardScaler()
    target_scaler.fit(train[['FVC']])
    train['FVC_scaled'] = target_scaler.transform(train[['FVC']])
    
    print(f"✅ Preprocessing Complete. Final Shape: {train.shape}")
    print(f"✅ Features Used: {len(feature_cols)}")
    
    return train, feature_cols, target_scaler

# ==========================================
# 3. MODEL
# ==========================================
class EnhancedQuantileMLP(nn.Module):
    def __init__(self, input_dim, quantiles, dropout=0.25):
        super().__init__()
        h1, h2, h3 = 256, 128, 64
        
        self.net = nn.Sequential(
            nn.Linear(input_dim, h1), nn.BatchNorm1d(h1), nn.LeakyReLU(0.1), nn.Dropout(dropout),
            nn.Linear(h1, h2), nn.BatchNorm1d(h2), nn.LeakyReLU(0.1), nn.Dropout(dropout),
            nn.Linear(h2, h3), nn.BatchNorm1d(h3), nn.LeakyReLU(0.1), nn.Dropout(dropout),
            nn.Linear(h3, len(quantiles))
        )
    
    def forward(self, x):
        return self.net(x)

def quantile_loss(preds, target, quantiles):
    assert not target.requires_grad
    losses = []
    for i, q in enumerate(quantiles):
        errors = target - preds[:, i].unsqueeze(1)
        loss = torch.max((q-1) * errors, q * errors)
        losses.append(loss)
    return torch.mean(torch.sum(torch.cat(losses, dim=1), dim=1))

# ==========================================
# 4. METRICS
# ==========================================
def calculate_metrics(y_true, q_preds_real):
    q20 = q_preds_real[:, 0]
    q50 = q_preds_real[:, 1]
    q80 = q_preds_real[:, 2]
    
    sigma = q80 - q20
    sigma_clipped = np.maximum(sigma, 70)
    
    delta = np.minimum(np.abs(y_true - q50), 1000)
    lll = - (np.sqrt(2) * delta / sigma_clipped) - np.log(np.sqrt(2) * sigma_clipped)
    
    mse = mean_squared_error(y_true, q50)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, q50)
    r2 = r2_score(y_true, q50)
    rmae = mae / (np.mean(np.abs(y_true)) + 1e-6)
    
    return {
        'lll': np.mean(lll),
        'mse': mse,
        'rmse': rmse,
        'mae': mae,
        'rmae': rmae,
        'r2': r2
    }

# ==========================================
# 5. 🧪 EXPERIMENT 1: BASELINE ANCHORING (FIXED)
# ==========================================
def apply_baseline_anchoring(df, predictions):
    """
    Fixed to avoid column name collisions (KeyError: Base_Week).
    """
    df_copy = df.copy()
    preds_copy = predictions.copy()
    
    # Calculate fresh baseline info with UNIQUE column names
    baseline_info = df_copy.groupby('Patient').agg({
        'Weeks': 'min',
        'FVC': 'first'
    }).reset_index()
    baseline_info.columns = ['Patient', 'Anchoring_Week', 'Anchoring_FVC']
    
    # Merge
    df_copy = df_copy.merge(baseline_info, on='Patient', how='left')
    
    patient_shifts = {}
    
    for patient in df_copy['Patient'].unique():
        patient_mask = df_copy['Patient'] == patient
        patient_data = df_copy[patient_mask]
        patient_preds = preds_copy[patient_mask]
        
        # Use the specific anchoring week column
        baseline_mask = patient_data['Weeks'] == patient_data['Anchoring_Week'].iloc[0]
        
        if baseline_mask.sum() > 0:
            baseline_pred = patient_preds[baseline_mask.values][0, 1]  # q50
            baseline_true = patient_data[baseline_mask]['FVC'].iloc[0] # Use original FVC
            shift = baseline_true - baseline_pred
        else:
            shift = 0
        
        patient_shifts[patient] = shift
    
    # Apply shifts
    for i in range(len(df_copy)):
        patient = df_copy.iloc[i]['Patient']
        shift = patient_shifts.get(patient, 0)
        preds_copy[i] += shift
    
    return preds_copy, patient_shifts

# ==========================================
# 6. 🧪 EXPERIMENT 2: SIGMA CALIBRATION
# ==========================================
def calibrate_sigma(predictions, scale_factor=0.85):
    preds_copy = predictions.copy()
    q20, q50, q80 = preds_copy[:, 0], preds_copy[:, 1], preds_copy[:, 2]
    
    new_q20 = q50 - (q50 - q20) * scale_factor
    new_q80 = q50 + (q80 - q50) * scale_factor
    
    return np.column_stack([new_q20, q50, new_q80])

# ==========================================
# 7. MAIN TRAINING LOOP
# ==========================================
def train_with_experiments():
    df, features, target_scaler = preprocess_data(CONFIG)
    patients = df['Patient'].unique()
    kf = KFold(n_splits=CONFIG['n_folds'], shuffle=True, random_state=CONFIG['seed'])
    
    oof_indices = []
    oof_raw_preds = []
    oof_trues = []
    
    print(f"\n🚀 Training with {len(df)} visits across {len(patients)} patients")
    print(f"📊 Features: {len(features)}")
    print("="*80)
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(patients)):
        print(f"\n{'='*80}")
        print(f"FOLD {fold+1}/{CONFIG['n_folds']}")
        print(f"{'='*80}")
        
        train_p, val_p = patients[train_idx], patients[val_idx]
        train_data = df[df['Patient'].isin(train_p)]
        val_data = df[df['Patient'].isin(val_p)]
        
        X_train = torch.tensor(train_data[features].values, dtype=torch.float32).to(CONFIG['device'])
        y_train_scaled = torch.tensor(train_data['FVC_scaled'].values, dtype=torch.float32).unsqueeze(1).to(CONFIG['device'])
        
        X_val = torch.tensor(val_data[features].values, dtype=torch.float32).to(CONFIG['device'])
        y_val_real = val_data['FVC'].values
        
        model = EnhancedQuantileMLP(len(features), CONFIG['quantiles']).to(CONFIG['device'])
        optimizer = optim.AdamW(model.parameters(), lr=CONFIG['lr'], weight_decay=CONFIG['weight_decay'])
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='max', factor=0.5, patience=20, verbose=False
        )
        
        best_lll = -float('inf')
        best_preds_real = None
        patience_counter = 0
        
        for epoch in range(CONFIG['epochs']):
            model.train()
            optimizer.zero_grad()
            preds_scaled = model(X_train)
            
            loss = quantile_loss(preds_scaled, y_train_scaled, CONFIG['quantiles'])
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            model.eval()
            with torch.no_grad():
                val_preds_scaled = model(X_val)
                val_preds_real = target_scaler.inverse_transform(val_preds_scaled.cpu().numpy())
                
                metrics = calculate_metrics(y_val_real, val_preds_real)
                lll = metrics['lll']
                
            scheduler.step(lll)
            
            if lll > best_lll:
                best_lll = lll
                best_preds_real = val_preds_real.copy()
                patience_counter = 0
            else:
                patience_counter += 1
                
            if patience_counter >= CONFIG['patience']:
                break
        
        oof_indices.extend(val_data.index.tolist())
        oof_raw_preds.append(best_preds_real)
        oof_trues.extend(y_val_real)
        
        fold_metrics = calculate_metrics(y_val_real, best_preds_real)
        print(f"\n📈 FOLD {fold+1} BASELINE RESULTS:")
        print(f"  R²:   {fold_metrics['r2']:.4f}")
        print(f"  RMSE: {fold_metrics['rmse']:.2f} mL")
        print(f"  LLL:  {fold_metrics['lll']:.4f}")
    
    # Combine OOF
    oof_raw_preds = np.vstack(oof_raw_preds)
    oof_trues = np.array(oof_trues)
    oof_df = df.loc[oof_indices].copy()
    
    print("\n" + "="*80)
    print("🏁 BASELINE (NO POST-PROCESSING)")
    print("="*80)
    baseline_metrics = calculate_metrics(oof_trues, oof_raw_preds)
    print(f"R²:   {baseline_metrics['r2']:.4f}   {'✅' if baseline_metrics['r2'] > 0.88 else '❌'} (Target > 0.88)")
    print(f"RMSE: {baseline_metrics['rmse']:.2f} mL {'✅' if baseline_metrics['rmse'] < 170 else '❌'} (Target < 170)")
    print(f"LLL:  {baseline_metrics['lll']:.4f}   {'✅' if baseline_metrics['lll'] > -6.64 else '❌'} (Target > -6.64)")
    
    # ==========================================
    # 🧪 EXPERIMENT 1: BASELINE ANCHORING
    # ==========================================
    print("\n" + "="*80)
    print("🧪 EXPERIMENT 1: BASELINE ANCHORING")
    print("="*80)
    
    # This function is now fixed!
    anchored_preds, shifts = apply_baseline_anchoring(oof_df, oof_raw_preds)
    anchored_metrics = calculate_metrics(oof_trues, anchored_preds)
    
    print(f"Average patient shift: {np.mean(list(shifts.values())):.2f} mL")
    print(f"\nR²:   {anchored_metrics['r2']:.4f}   {'✅' if anchored_metrics['r2'] > 0.88 else '❌'}")
    print(f"RMSE: {anchored_metrics['rmse']:.2f} mL {'✅' if anchored_metrics['rmse'] < 170 else '❌'} (Δ: {anchored_metrics['rmse'] - baseline_metrics['rmse']:+.2f})")
    print(f"LLL:  {anchored_metrics['lll']:.4f}   {'✅' if anchored_metrics['lll'] > -6.64 else '❌'} (Δ: {anchored_metrics['lll'] - baseline_metrics['lll']:+.4f})")
    
    # ==========================================
    # 🧪 EXPERIMENT 2: SIGMA CALIBRATION
    # ==========================================
    print("\n" + "="*80)
    print("🧪 EXPERIMENT 2: SIGMA CALIBRATION (After Anchoring)")
    print("="*80)
    
    best_lll = anchored_metrics['lll']
    best_factor = 1.0
    
    for scale in [0.95, 0.90, 0.85, 0.80, 0.75, 0.70]:
        calibrated = calibrate_sigma(anchored_preds, scale)
        metrics = calculate_metrics(oof_trues, calibrated)
        
        improvement = "✅" if metrics['lll'] > best_lll else ""
        print(f"  σ×{scale:.2f} → LLL: {metrics['lll']:.4f} (RMSE: {metrics['rmse']:.2f}) {improvement}")
        
        if metrics['lll'] > best_lll:
            best_lll = metrics['lll']
            best_factor = scale
    
    # Apply best sigma calibration
    final_preds = calibrate_sigma(anchored_preds, best_factor)
    final_metrics = calculate_metrics(oof_trues, final_preds)
    
    print("\n" + "="*80)
    print("🏆 FINAL RESULTS (Anchoring + Best Sigma)")
    print("="*80)
    print(f"Best σ scale factor: {best_factor:.2f}")
    print(f"\nR²:   {final_metrics['r2']:.4f}   {'✅' if final_metrics['r2'] > 0.88 else '❌'} (Target > 0.88)")
    print(f"RMSE: {final_metrics['rmse']:.2f} mL {'✅' if final_metrics['rmse'] < 170 else '❌'} (Target < 170)")
    print(f"LLL:  {final_metrics['lll']:.4f}   {'✅' if final_metrics['lll'] > -6.64 else '❌'} (Target > -6.64)")

if __name__ == "__main__":
    train_with_experiments()

✅ Preprocessing Complete. Final Shape: (1549, 26)
✅ Features Used: 19

🚀 Training with 1549 visits across 176 patients
📊 Features: 19

FOLD 1/5

📈 FOLD 1 BASELINE RESULTS:
  R²:   0.9070
  RMSE: 239.89 mL
  LLL:  -6.8198

FOLD 2/5

📈 FOLD 2 BASELINE RESULTS:
  R²:   0.9180
  RMSE: 255.34 mL
  LLL:  -6.9550

FOLD 3/5

📈 FOLD 3 BASELINE RESULTS:
  R²:   0.8814
  RMSE: 259.07 mL
  LLL:  -6.8752

FOLD 4/5

📈 FOLD 4 BASELINE RESULTS:
  R²:   0.8966
  RMSE: 246.11 mL
  LLL:  -6.9504

FOLD 5/5

📈 FOLD 5 BASELINE RESULTS:
  R²:   0.9387
  RMSE: 224.47 mL
  LLL:  -6.8886

🏁 BASELINE (NO POST-PROCESSING)
R²:   0.9132   ✅ (Target > 0.88)
RMSE: 245.23 mL ❌ (Target < 170)
LLL:  -6.8973   ❌ (Target > -6.64)

🧪 EXPERIMENT 1: BASELINE ANCHORING
Average patient shift: 14.84 mL

R²:   0.9292   ✅
RMSE: 221.49 mL ❌ (Δ: -23.73)
LLL:  -6.7673   ❌ (Δ: +0.1300)

🧪 EXPERIMENT 2: SIGMA CALIBRATION (After Anchoring)
  σ×0.95 → LLL: -6.7501 (RMSE: 221.49) ✅
  σ×0.90 → LLL: -6.7338 (RMSE: 221.49) ✅
  σ×0.85 → LLL:

## The Diagnosis: You have solved the Disease, but not the Noise.
## Strategy C
### Deploy Strategy C: Ratio Prediction Architecture
Why this will work:

#### Mathematical Constraint: 
By predicting FVC_ratio = FVC_current / FVC_baseline, you force the model to output values near 1.0 (range: 0.7-1.1 instead of 1000-6000)
#### Automatic Anchoring: 
At inference, Predicted_FVC = Predicted_Ratio × True_Baseline_FVC - this mechanically eliminates baseline error
#### Smaller Loss Surface: 
Predicting a ratio (normalized target) is a simpler optimization problem

In [15]:
import os
import pandas as pd
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings

warnings.filterwarnings('ignore')

# ==========================================
# CONFIGURATION
# ==========================================
CONFIG = {
    "lr": 2e-3,
    "weight_decay": 3e-5,
    "batch_size": 64,
    "epochs": 300,
    "n_folds": 5,
    "quantiles": [0.2, 0.5, 0.8], 
    "patience": 50,
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    "data_dir": "../input/osic-pulmonary-fibrosis-progression",
    "biomarker_path": "../input/feature-extraction-u-net-segmentation/master_dataset.csv",
    "seed": 42
}

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(CONFIG['seed'])

# ==========================================
# STRATEGY C: RATIO-BASED DATA PREPROCESSING
# ==========================================
def preprocess_data_ratio(config):
    """
    KEY CHANGE: Target is now FVC_RATIO = Current_FVC / Baseline_FVC
    This normalizes all patients to start at 1.0 and learn decay rates.
    """
    clinical_df = pd.read_csv(f"{config['data_dir']}/train.csv")
    biomarkers_df = pd.read_csv(config['biomarker_path'])
    
    image_features = [
        'lung_vol_ml', 'hu_mean', 'hu_std', 'hu_skew', 'hu_kurt',
        'glcm_contrast', 'glcm_homogeneity', 'glcm_energy', 'glcm_correlation'
    ]
    
    cols_to_keep = ['Patient'] + [c for c in image_features if c in biomarkers_df.columns]
    biomarkers_clean = biomarkers_df[cols_to_keep].drop_duplicates(subset=['Patient'])
    
    train = clinical_df.merge(biomarkers_clean, on='Patient', how='inner')
    
    train['Weeks'] = train['Weeks'].astype(int)
    train.sort_values(['Patient', 'Weeks'], inplace=True)
    
    # Get baseline FVC for each patient
    baseline = train.groupby('Patient').first().reset_index()
    baseline = baseline[['Patient', 'FVC', 'Percent']].rename(
        columns={'FVC': 'Base_FVC', 'Percent': 'Base_Percent'}
    )
    train = train.merge(baseline, on='Patient', how='left')
    
    base_weeks = train.groupby('Patient')['Weeks'].min().reset_index().rename(
        columns={'Weeks': 'Base_Week'}
    )
    train = train.merge(base_weeks, on='Patient', how='left')
    train['Relative_Weeks'] = train['Weeks'] - train['Base_Week']
    
    # ============================================
    # 🔥 CRITICAL CHANGE: CREATE RATIO TARGET
    # ============================================
    train['FVC_Ratio'] = train['FVC'] / train['Base_FVC']
    
    # Sanity check: Most ratios should be 0.7-1.1 (fibrosis causes decline)
    print(f"📊 FVC_Ratio Distribution:")
    print(f"   Mean: {train['FVC_Ratio'].mean():.3f}")
    print(f"   Std:  {train['FVC_Ratio'].std():.3f}")
    print(f"   Min:  {train['FVC_Ratio'].min():.3f}")
    print(f"   Max:  {train['FVC_Ratio'].max():.3f}")
    
    # Interaction features (same as before)
    train['FVC_Week_Interaction'] = train['Base_FVC'] * train['Relative_Weeks']
    train['Age_Week_Interaction'] = train['Age'] * train['Relative_Weeks']
    
    available_img_feats = [c for c in image_features if c in train.columns]
    
    if 'lung_vol_ml' in train.columns:
        train['LungVol_FVC_Ratio'] = train['lung_vol_ml'] / (train['Base_FVC'] + 1e-6)
    
    # Feature scaling (EXCLUDE Base_FVC from scaling - we need its real value at inference)
    scaler = StandardScaler()
    
    num_cols = ['Age', 'Base_Percent', 'Relative_Weeks'] + available_img_feats
    interaction_cols = ['FVC_Week_Interaction', 'Age_Week_Interaction']
    
    if 'LungVol_FVC_Ratio' in train.columns:
        interaction_cols.append('LungVol_FVC_Ratio')
    
    all_num_cols = num_cols + interaction_cols
    train[all_num_cols] = scaler.fit_transform(train[all_num_cols])
    
    # Keep Base_FVC UNSCALED for reconstruction
    train['Base_FVC_Raw'] = train['Base_FVC']
    
    train['Sex'] = train['Sex'].apply(lambda x: 1 if x == 'Male' else 0)
    train['Smk_Ex'] = train['SmokingStatus'].apply(lambda x: 1 if x == 'Ex-smoker' else 0)
    train['Smk_Cur'] = train['SmokingStatus'].apply(lambda x: 1 if x == 'Currently smokes' else 0)
    
    # Feature list (Base_FVC is now RAW, not scaled)
    feature_cols = all_num_cols + ['Base_FVC_Raw', 'Sex', 'Smk_Ex', 'Smk_Cur']
    
    # Target scaling (RATIO instead of raw FVC)
    ratio_scaler = StandardScaler()
    ratio_scaler.fit(train[['FVC_Ratio']])
    train['FVC_Ratio_Scaled'] = ratio_scaler.transform(train[['FVC_Ratio']])
    
    print(f"✅ Preprocessing Complete. Final Shape: {train.shape}")
    print(f"✅ Features Used: {len(feature_cols)}")
    
    return train, feature_cols, ratio_scaler

# ==========================================
# MODEL (Same architecture)
# ==========================================
class EnhancedQuantileMLP(nn.Module):
    def __init__(self, input_dim, quantiles, dropout=0.25):
        super().__init__()
        h1, h2, h3 = 256, 128, 64
        
        self.net = nn.Sequential(
            nn.Linear(input_dim, h1),
            nn.BatchNorm1d(h1),
            nn.LeakyReLU(0.1),
            nn.Dropout(dropout),
            
            nn.Linear(h1, h2),
            nn.BatchNorm1d(h2),
            nn.LeakyReLU(0.1),
            nn.Dropout(dropout),
            
            nn.Linear(h2, h3),
            nn.BatchNorm1d(h3),
            nn.LeakyReLU(0.1),
            nn.Dropout(dropout),
            
            nn.Linear(h3, len(quantiles))
        )
    
    def forward(self, x):
        return self.net(x)

def quantile_loss(preds, target, quantiles):
    assert not target.requires_grad
    losses = []
    for i, q in enumerate(quantiles):
        errors = target - preds[:, i].unsqueeze(1)
        loss = torch.max((q-1) * errors, q * errors)
        losses.append(loss)
    return torch.mean(torch.sum(torch.cat(losses, dim=1), dim=1))

# ==========================================
# METRICS (Now convert ratios back to FVC)
# ==========================================
def calculate_metrics(y_true_fvc, q_preds_ratio, baseline_fvc):
    """
    KEY CHANGE: Predictions are ratios, must multiply by baseline to get FVC.
    
    Args:
        y_true_fvc: True FVC values (mL)
        q_preds_ratio: Predicted quantiles as RATIOS
        baseline_fvc: Baseline FVC for each sample (mL)
    """
    # Reconstruct FVC predictions from ratios
    q20 = q_preds_ratio[:, 0] * baseline_fvc
    q50 = q_preds_ratio[:, 1] * baseline_fvc
    q80 = q_preds_ratio[:, 2] * baseline_fvc
    
    sigma = q80 - q20
    sigma_clipped = np.maximum(sigma, 70)
    
    delta = np.minimum(np.abs(y_true_fvc - q50), 1000)
    lll = - (np.sqrt(2) * delta / sigma_clipped) - np.log(np.sqrt(2) * sigma_clipped)
    
    mse = mean_squared_error(y_true_fvc, q50)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true_fvc, q50)
    r2 = r2_score(y_true_fvc, q50)
    rmae = mae / (np.mean(np.abs(y_true_fvc)) + 1e-6)
    
    return {
        'lll': np.mean(lll),
        'mse': mse,
        'rmse': rmse,
        'mae': mae,
        'rmae': rmae,
        'r2': r2
    }

# ==========================================
# SIGMA CALIBRATION (Optional post-processing)
# ==========================================
def calibrate_sigma_ratio(ratio_preds, scale_factor=0.85):
    """Compress uncertainty on ratio predictions."""
    preds_copy = ratio_preds.copy()
    q20, q50, q80 = preds_copy[:, 0], preds_copy[:, 1], preds_copy[:, 2]
    
    new_q20 = q50 - (q50 - q20) * scale_factor
    new_q80 = q50 + (q80 - q50) * scale_factor
    
    return np.column_stack([new_q20, q50, new_q80])

# ==========================================
# TRAINING WITH RATIO PREDICTION
# ==========================================
def train_ratio_model():
    df, features, ratio_scaler = preprocess_data_ratio(CONFIG)
    patients = df['Patient'].unique()
    kf = KFold(n_splits=CONFIG['n_folds'], shuffle=True, random_state=CONFIG['seed'])
    
    oof_indices = []
    oof_ratio_preds = []
    oof_trues_fvc = []
    oof_baselines = []
    
    print(f"\n🚀 Training RATIO PREDICTION Model")
    print(f"📊 Training on {len(df)} visits across {len(patients)} patients")
    print(f"🎯 Target: FVC_Ratio = Current_FVC / Baseline_FVC")
    print("="*80)
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(patients)):
        print(f"\n{'='*80}")
        print(f"FOLD {fold+1}/{CONFIG['n_folds']}")
        print(f"{'='*80}")
        
        train_p, val_p = patients[train_idx], patients[val_idx]
        train_data = df[df['Patient'].isin(train_p)]
        val_data = df[df['Patient'].isin(val_p)]
        
        X_train = torch.tensor(train_data[features].values, dtype=torch.float32).to(CONFIG['device'])
        y_train_ratio_scaled = torch.tensor(train_data['FVC_Ratio_Scaled'].values, dtype=torch.float32).unsqueeze(1).to(CONFIG['device'])
        
        X_val = torch.tensor(val_data[features].values, dtype=torch.float32).to(CONFIG['device'])
        y_val_fvc = val_data['FVC'].values
        val_baselines = val_data['Base_FVC_Raw'].values
        
        model = EnhancedQuantileMLP(len(features), CONFIG['quantiles']).to(CONFIG['device'])
        optimizer = optim.AdamW(model.parameters(), lr=CONFIG['lr'], weight_decay=CONFIG['weight_decay'])
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='max', factor=0.5, patience=20, verbose=False
        )
        
        best_lll = -float('inf')
        best_ratio_preds = None
        patience_counter = 0
        
        for epoch in range(CONFIG['epochs']):
            model.train()
            optimizer.zero_grad()
            preds_ratio_scaled = model(X_train)
            
            loss = quantile_loss(preds_ratio_scaled, y_train_ratio_scaled, CONFIG['quantiles'])
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            model.eval()
            with torch.no_grad():
                val_preds_ratio_scaled = model(X_val)
                val_preds_ratio = ratio_scaler.inverse_transform(val_preds_ratio_scaled.cpu().numpy())
                
                metrics = calculate_metrics(y_val_fvc, val_preds_ratio, val_baselines)
                lll = metrics['lll']
                
            scheduler.step(lll)
            
            if lll > best_lll:
                best_lll = lll
                best_ratio_preds = val_preds_ratio.copy()
                patience_counter = 0
            else:
                patience_counter += 1
                
            if patience_counter >= CONFIG['patience']:
                break
        
        oof_indices.extend(val_data.index.tolist())
        oof_ratio_preds.append(best_ratio_preds)
        oof_trues_fvc.extend(y_val_fvc)
        oof_baselines.extend(val_baselines)
        
        fold_metrics = calculate_metrics(y_val_fvc, best_ratio_preds, val_baselines)
        
        print(f"\n📈 FOLD {fold+1} RESULTS:")
        print(f"  R²:   {fold_metrics['r2']:.4f}")
        print(f"  RMSE: {fold_metrics['rmse']:.2f} mL")
        print(f"  MAE:  {fold_metrics['mae']:.2f} mL")
        print(f"  LLL:  {fold_metrics['lll']:.4f}")
    
    oof_ratio_preds = np.vstack(oof_ratio_preds)
    oof_trues_fvc = np.array(oof_trues_fvc)
    oof_baselines = np.array(oof_baselines)
    
    print("\n" + "="*80)
    print("🏁 BASELINE RATIO PREDICTION RESULTS")
    print("="*80)
    baseline_metrics = calculate_metrics(oof_trues_fvc, oof_ratio_preds, oof_baselines)
    print(f"R²:   {baseline_metrics['r2']:.4f}   {'✅' if baseline_metrics['r2'] > 0.88 else '❌'} (Target > 0.88)")
    print(f"RMSE: {baseline_metrics['rmse']:.2f} mL {'✅' if baseline_metrics['rmse'] < 170 else '❌'} (Target < 170)")
    print(f"MAE:  {baseline_metrics['mae']:.2f} mL")
    print(f"LLL:  {baseline_metrics['lll']:.4f}   {'✅' if baseline_metrics['lll'] > -6.64 else '❌'} (Target > -6.64)")
    
    # Optional: Test sigma calibration
    print("\n" + "="*80)
    print("🧪 OPTIONAL: SIGMA CALIBRATION")
    print("="*80)
    
    best_lll = baseline_metrics['lll']
    best_factor = 1.0
    
    for scale in [0.95, 0.90, 0.85, 0.80, 0.75]:
        calibrated = calibrate_sigma_ratio(oof_ratio_preds, scale)
        metrics = calculate_metrics(oof_trues_fvc, calibrated, oof_baselines)
        
        improvement = "✅" if metrics['lll'] > best_lll else ""
        print(f"  σ×{scale:.2f} → LLL: {metrics['lll']:.4f} (RMSE: {metrics['rmse']:.2f}) {improvement}")
        
        if metrics['lll'] > best_lll:
            best_lll = metrics['lll']
            best_factor = scale
    
    if best_factor < 1.0:
        final_preds = calibrate_sigma_ratio(oof_ratio_preds, best_factor)
        final_metrics = calculate_metrics(oof_trues_fvc, final_preds, oof_baselines)
        
        print("\n" + "="*80)
        print("🏆 FINAL RESULTS (With Optimal Sigma)")
        print("="*80)
        print(f"Best σ scale factor: {best_factor:.2f}")
        print(f"\nR²:   {final_metrics['r2']:.4f}   {'✅' if final_metrics['r2'] > 0.88 else '❌'} (Target > 0.88)")
        print(f"RMSE: {final_metrics['rmse']:.2f} mL {'✅' if final_metrics['rmse'] < 170 else '❌'} (Target < 170)")
        print(f"MAE:  {final_metrics['mae']:.2f} mL")
        print(f"LLL:  {final_metrics['lll']:.4f}   {'✅' if final_metrics['lll'] > -6.64 else '❌'} (Target > -6.64)")
        
        print("\n" + "="*80)
        print("📊 IMPROVEMENT OVER BASELINE")
        print("="*80)
        print(f"RMSE: {baseline_metrics['rmse']:.2f} → {final_metrics['rmse']:.2f} ({final_metrics['rmse'] - baseline_metrics['rmse']:+.2f} mL)")
        print(f"LLL:  {baseline_metrics['lll']:.4f} → {final_metrics['lll']:.4f} ({final_metrics['lll'] - baseline_metrics['lll']:+.4f})")
    else:
        print("\n✅ No sigma calibration needed - baseline predictions are optimal!")
    
    print("="*80)

if __name__ == "__main__":
    train_ratio_model()

📊 FVC_Ratio Distribution:
   Mean: 0.967
   Std:  0.086
   Min:  0.461
   Max:  1.288
✅ Preprocessing Complete. Final Shape: (1549, 28)
✅ Features Used: 19

🚀 Training RATIO PREDICTION Model
📊 Training on 1549 visits across 176 patients
🎯 Target: FVC_Ratio = Current_FVC / Baseline_FVC

FOLD 1/5

📈 FOLD 1 RESULTS:
  R²:   0.9355
  RMSE: 199.69 mL
  MAE:  130.93 mL
  LLL:  -6.5190

FOLD 2/5

📈 FOLD 2 RESULTS:
  R²:   0.9414
  RMSE: 215.84 mL
  MAE:  151.09 mL
  LLL:  -6.6674

FOLD 3/5

📈 FOLD 3 RESULTS:
  R²:   0.9052
  RMSE: 231.61 mL
  MAE:  159.39 mL
  LLL:  -6.7766

FOLD 4/5

📈 FOLD 4 RESULTS:
  R²:   0.9227
  RMSE: 212.80 mL
  MAE:  147.48 mL
  LLL:  -6.6250

FOLD 5/5

📈 FOLD 5 RESULTS:
  R²:   0.9511
  RMSE: 200.52 mL
  MAE:  132.51 mL
  LLL:  -6.5309

🏁 BASELINE RATIO PREDICTION RESULTS
R²:   0.9349   ✅ (Target > 0.88)
RMSE: 212.34 mL ❌ (Target < 170)
MAE:  144.19 mL
LLL:  -6.6231   ✅ (Target > -6.64)

🧪 OPTIONAL: SIGMA CALIBRATION
  σ×0.95 → LLL: -6.6207 (RMSE: 212.34) ✅
  σ×0.90

## Action Plan:

* Save your OOF predictions from this run (oof_ratio_preds and oof_trues_fvc).

* Train a Gradient Boosting Model (LightGBM/XGBoost) on the exact same dataset (master_dataset.csv).

* Blend: Average the predictions: Final = 0.6 * NeuralNet + 0.4 * LightGBM.

* GBMs are excellent at finding "edge cases" (like Fold 3) that Neural Nets smooth over.

This typically drops RMSE by another 5-10 points.

In [26]:
import os
import pandas as pd
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings

warnings.filterwarnings('ignore')

# ==========================================
# CONFIGURATION
# ==========================================
CONFIG = {
    "lr": 2e-3,
    "weight_decay": 3e-5,
    "batch_size": 64,
    "epochs": 300,
    "n_folds": 5,
    "quantiles": [0.2, 0.5, 0.8], 
    "patience": 50,
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    "data_dir": "../input/osic-pulmonary-fibrosis-progression",
    "biomarker_path": "../input/feature-extraction-u-net-segmentation/master_dataset.csv",
    "seed": 42
}

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(CONFIG['seed'])

# ==========================================
# STRATEGY C: RATIO-BASED DATA PREPROCESSING
# ==========================================
def preprocess_data_ratio(config):
    """
    KEY CHANGE: Target is now FVC_RATIO = Current_FVC / Baseline_FVC
    This normalizes all patients to start at 1.0 and learn decay rates.
    """
    clinical_df = pd.read_csv(f"{config['data_dir']}/train.csv")
    biomarkers_df = pd.read_csv(config['biomarker_path'])
    
    image_features = [
        'lung_vol_ml', 'hu_mean', 'hu_std', 'hu_skew', 'hu_kurt',
        'glcm_contrast', 'glcm_homogeneity', 'glcm_energy', 'glcm_correlation'
    ]
    
    cols_to_keep = ['Patient'] + [c for c in image_features if c in biomarkers_df.columns]
    biomarkers_clean = biomarkers_df[cols_to_keep].drop_duplicates(subset=['Patient'])
    
    train = clinical_df.merge(biomarkers_clean, on='Patient', how='inner')
    
    train['Weeks'] = train['Weeks'].astype(int)
    train.sort_values(['Patient', 'Weeks'], inplace=True)
    
    # Get baseline FVC for each patient
    baseline = train.groupby('Patient').first().reset_index()
    baseline = baseline[['Patient', 'FVC', 'Percent']].rename(
        columns={'FVC': 'Base_FVC', 'Percent': 'Base_Percent'}
    )
    train = train.merge(baseline, on='Patient', how='left')
    
    base_weeks = train.groupby('Patient')['Weeks'].min().reset_index().rename(
        columns={'Weeks': 'Base_Week'}
    )
    train = train.merge(base_weeks, on='Patient', how='left')
    train['Relative_Weeks'] = train['Weeks'] - train['Base_Week']
    
    # ============================================
    # 🔥 CRITICAL CHANGE: CREATE RATIO TARGET
    # ============================================
    train['FVC_Ratio'] = train['FVC'] / train['Base_FVC']
    
    # Sanity check: Most ratios should be 0.7-1.1 (fibrosis causes decline)
    print(f"📊 FVC_Ratio Distribution:")
    print(f"   Mean: {train['FVC_Ratio'].mean():.3f}")
    print(f"   Std:  {train['FVC_Ratio'].std():.3f}")
    print(f"   Min:  {train['FVC_Ratio'].min():.3f}")
    print(f"   Max:  {train['FVC_Ratio'].max():.3f}")
    
    # Interaction features (same as before)
    train['FVC_Week_Interaction'] = train['Base_FVC'] * train['Relative_Weeks']
    train['Age_Week_Interaction'] = train['Age'] * train['Relative_Weeks']
    
    available_img_feats = [c for c in image_features if c in train.columns]
    
    if 'lung_vol_ml' in train.columns:
        train['LungVol_FVC_Ratio'] = train['lung_vol_ml'] / (train['Base_FVC'] + 1e-6)
    
    # Feature scaling (EXCLUDE Base_FVC from scaling - we need its real value at inference)
    scaler = StandardScaler()
    
    num_cols = ['Age', 'Base_Percent', 'Relative_Weeks'] + available_img_feats
    interaction_cols = ['FVC_Week_Interaction', 'Age_Week_Interaction']
    
    if 'LungVol_FVC_Ratio' in train.columns:
        interaction_cols.append('LungVol_FVC_Ratio')
    
    all_num_cols = num_cols + interaction_cols
    train[all_num_cols] = scaler.fit_transform(train[all_num_cols])
    
    # Keep Base_FVC UNSCALED for reconstruction
    train['Base_FVC_Raw'] = train['Base_FVC']
    
    train['Sex'] = train['Sex'].apply(lambda x: 1 if x == 'Male' else 0)
    train['Smk_Ex'] = train['SmokingStatus'].apply(lambda x: 1 if x == 'Ex-smoker' else 0)
    train['Smk_Cur'] = train['SmokingStatus'].apply(lambda x: 1 if x == 'Currently smokes' else 0)
    
    # Feature list (Base_FVC is now RAW, not scaled)
    feature_cols = all_num_cols + ['Base_FVC_Raw', 'Sex', 'Smk_Ex', 'Smk_Cur']
    
    # Target scaling (RATIO instead of raw FVC)
    ratio_scaler = StandardScaler()
    ratio_scaler.fit(train[['FVC_Ratio']])
    train['FVC_Ratio_Scaled'] = ratio_scaler.transform(train[['FVC_Ratio']])
    
    print(f"✅ Preprocessing Complete. Final Shape: {train.shape}")
    print(f"✅ Features Used: {len(feature_cols)}")
    
    return train, feature_cols, ratio_scaler

# ==========================================
# MODEL (Same architecture)
# ==========================================
class EnhancedQuantileMLP(nn.Module):
    def __init__(self, input_dim, quantiles, dropout=0.25):
        super().__init__()
        h1, h2, h3 = 256, 128, 64
        
        self.net = nn.Sequential(
            nn.Linear(input_dim, h1),
            nn.BatchNorm1d(h1),
            nn.LeakyReLU(0.1),
            nn.Dropout(dropout),
            
            nn.Linear(h1, h2),
            nn.BatchNorm1d(h2),
            nn.LeakyReLU(0.1),
            nn.Dropout(dropout),
            
            nn.Linear(h2, h3),
            nn.BatchNorm1d(h3),
            nn.LeakyReLU(0.1),
            nn.Dropout(dropout),
            
            nn.Linear(h3, len(quantiles))
        )
    
    def forward(self, x):
        return self.net(x)

def quantile_loss(preds, target, quantiles):
    assert not target.requires_grad
    losses = []
    for i, q in enumerate(quantiles):
        errors = target - preds[:, i].unsqueeze(1)
        loss = torch.max((q-1) * errors, q * errors)
        losses.append(loss)
    return torch.mean(torch.sum(torch.cat(losses, dim=1), dim=1))

# ==========================================
# METRICS (Now convert ratios back to FVC)
# ==========================================
def calculate_metrics(y_true_fvc, q_preds_ratio, baseline_fvc):
    """
    KEY CHANGE: Predictions are ratios, must multiply by baseline to get FVC.
    
    Args:
        y_true_fvc: True FVC values (mL)
        q_preds_ratio: Predicted quantiles as RATIOS
        baseline_fvc: Baseline FVC for each sample (mL)
    """
    # Reconstruct FVC predictions from ratios
    q20 = q_preds_ratio[:, 0] * baseline_fvc
    q50 = q_preds_ratio[:, 1] * baseline_fvc
    q80 = q_preds_ratio[:, 2] * baseline_fvc
    
    sigma = q80 - q20
    sigma_clipped = np.maximum(sigma, 70)
    
    delta = np.minimum(np.abs(y_true_fvc - q50), 1000)
    lll = - (np.sqrt(2) * delta / sigma_clipped) - np.log(np.sqrt(2) * sigma_clipped)
    
    mse = mean_squared_error(y_true_fvc, q50)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true_fvc, q50)
    r2 = r2_score(y_true_fvc, q50)
    rmae = mae / (np.mean(np.abs(y_true_fvc)) + 1e-6)
    
    return {
        'lll': np.mean(lll),
        'mse': mse,
        'rmse': rmse,
        'mae': mae,
        'rmae': rmae,
        'r2': r2
    }

# ==========================================
# SIGMA CALIBRATION (Optional post-processing)
# ==========================================
def calibrate_sigma_ratio(ratio_preds, scale_factor=0.85):
    """Compress uncertainty on ratio predictions."""
    preds_copy = ratio_preds.copy()
    q20, q50, q80 = preds_copy[:, 0], preds_copy[:, 1], preds_copy[:, 2]
    
    new_q20 = q50 - (q50 - q20) * scale_factor
    new_q80 = q50 + (q80 - q50) * scale_factor
    
    return np.column_stack([new_q20, q50, new_q80])

# ==========================================
# TRAINING WITH RATIO PREDICTION
# ==========================================
def train_ratio_model():
    df, features, ratio_scaler = preprocess_data_ratio(CONFIG)
    patients = df['Patient'].unique()
    kf = KFold(n_splits=CONFIG['n_folds'], shuffle=True, random_state=CONFIG['seed'])
    
    oof_indices = []
    oof_ratio_preds = []
    oof_trues_fvc = []
    oof_baselines = []
    
    print(f"\n🚀 Training RATIO PREDICTION Model")
    print(f"📊 Training on {len(df)} visits across {len(patients)} patients")
    print(f"🎯 Target: FVC_Ratio = Current_FVC / Baseline_FVC")
    print("="*80)
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(patients)):
        print(f"\n{'='*80}")
        print(f"FOLD {fold+1}/{CONFIG['n_folds']}")
        print(f"{'='*80}")
        
        train_p, val_p = patients[train_idx], patients[val_idx]
        train_data = df[df['Patient'].isin(train_p)]
        val_data = df[df['Patient'].isin(val_p)]
        
        X_train = torch.tensor(train_data[features].values, dtype=torch.float32).to(CONFIG['device'])
        y_train_ratio_scaled = torch.tensor(train_data['FVC_Ratio_Scaled'].values, dtype=torch.float32).unsqueeze(1).to(CONFIG['device'])
        
        X_val = torch.tensor(val_data[features].values, dtype=torch.float32).to(CONFIG['device'])
        y_val_fvc = val_data['FVC'].values
        val_baselines = val_data['Base_FVC_Raw'].values
        
        model = EnhancedQuantileMLP(len(features), CONFIG['quantiles']).to(CONFIG['device'])
        optimizer = optim.AdamW(model.parameters(), lr=CONFIG['lr'], weight_decay=CONFIG['weight_decay'])
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='max', factor=0.5, patience=20, verbose=False
        )
        
        best_lll = -float('inf')
        best_ratio_preds = None
        patience_counter = 0
        
        for epoch in range(CONFIG['epochs']):
            model.train()
            optimizer.zero_grad()
            preds_ratio_scaled = model(X_train)
            
            loss = quantile_loss(preds_ratio_scaled, y_train_ratio_scaled, CONFIG['quantiles'])
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            model.eval()
            with torch.no_grad():
                val_preds_ratio_scaled = model(X_val)
                val_preds_ratio = ratio_scaler.inverse_transform(val_preds_ratio_scaled.cpu().numpy())
                
                metrics = calculate_metrics(y_val_fvc, val_preds_ratio, val_baselines)
                lll = metrics['lll']
                
            scheduler.step(lll)
            
            if lll > best_lll:
                best_lll = lll
                best_ratio_preds = val_preds_ratio.copy()
                patience_counter = 0
            else:
                patience_counter += 1
                
            if patience_counter >= CONFIG['patience']:
                break
        
        oof_indices.extend(val_data.index.tolist())
        oof_ratio_preds.append(best_ratio_preds)
        oof_trues_fvc.extend(y_val_fvc)
        oof_baselines.extend(val_baselines)
        
        fold_metrics = calculate_metrics(y_val_fvc, best_ratio_preds, val_baselines)
        
        print(f"\n📈 FOLD {fold+1} RESULTS:")
        print(f"  R²:   {fold_metrics['r2']:.4f}")
        print(f"  RMSE: {fold_metrics['rmse']:.2f} mL")
        print(f"  MAE:  {fold_metrics['mae']:.2f} mL")
        print(f"  LLL:  {fold_metrics['lll']:.4f}")
    
    oof_ratio_preds = np.vstack(oof_ratio_preds)
    oof_trues_fvc = np.array(oof_trues_fvc)
    oof_baselines = np.array(oof_baselines)
    
    print("\n" + "="*80)
    print("🏁 BASELINE RATIO PREDICTION RESULTS")
    print("="*80)
    baseline_metrics = calculate_metrics(oof_trues_fvc, oof_ratio_preds, oof_baselines)
    print(f"R²:   {baseline_metrics['r2']:.4f}   {'✅' if baseline_metrics['r2'] > 0.88 else '❌'} (Target > 0.88)")
    print(f"RMSE: {baseline_metrics['rmse']:.2f} mL {'✅' if baseline_metrics['rmse'] < 170 else '❌'} (Target < 170)")
    print(f"MAE:  {baseline_metrics['mae']:.2f} mL")
    print(f"LLL:  {baseline_metrics['lll']:.4f}   {'✅' if baseline_metrics['lll'] > -6.64 else '❌'} (Target > -6.64)")
    
    # Optional: Test sigma calibration
    print("\n" + "="*80)
    print("🧪 OPTIONAL: SIGMA CALIBRATION")
    print("="*80)
    
    best_lll = baseline_metrics['lll']
    best_factor = 1.0
    
    for scale in [0.95, 0.90, 0.85, 0.80, 0.75]:
        calibrated = calibrate_sigma_ratio(oof_ratio_preds, scale)
        metrics = calculate_metrics(oof_trues_fvc, calibrated, oof_baselines)
        
        improvement = "✅" if metrics['lll'] > best_lll else ""
        print(f"  σ×{scale:.2f} → LLL: {metrics['lll']:.4f} (RMSE: {metrics['rmse']:.2f}) {improvement}")
        
        if metrics['lll'] > best_lll:
            best_lll = metrics['lll']
            best_factor = scale
            torch.save(model.state_dict(), f"optimised_model_fold{fold+1}.pth")
    
    if best_factor < 1.0:
        final_preds = calibrate_sigma_ratio(oof_ratio_preds, best_factor)
        final_metrics = calculate_metrics(oof_trues_fvc, final_preds, oof_baselines)
        
        print("\n" + "="*80)
        print("🏆 FINAL RESULTS (With Optimal Sigma)")
        print("="*80)
        print(f"Best σ scale factor: {best_factor:.2f}")
        print(f"\nR²:   {final_metrics['r2']:.4f}   {'✅' if final_metrics['r2'] > 0.88 else '❌'} (Target > 0.88)")
        print(f"RMSE: {final_metrics['rmse']:.2f} mL {'✅' if final_metrics['rmse'] < 170 else '❌'} (Target < 170)")
        print(f"MAE:  {final_metrics['mae']:.2f} mL")
        print(f"LLL:  {final_metrics['lll']:.4f}   {'✅' if final_metrics['lll'] > -6.64 else '❌'} (Target > -6.64)")
        
        print("\n" + "="*80)
        print("📊 IMPROVEMENT OVER BASELINE")
        print("="*80)
        print(f"RMSE: {baseline_metrics['rmse']:.2f} → {final_metrics['rmse']:.2f} ({final_metrics['rmse'] - baseline_metrics['rmse']:+.2f} mL)")
        print(f"LLL:  {baseline_metrics['lll']:.4f} → {final_metrics['lll']:.4f} ({final_metrics['lll'] - baseline_metrics['lll']:+.4f})")
    else:
        print("\n✅ No sigma calibration needed - baseline predictions are optimal!")
    
    print("="*80)

if __name__ == "__main__":
    train_ratio_model()

📊 FVC_Ratio Distribution:
   Mean: 0.967
   Std:  0.086
   Min:  0.461
   Max:  1.288
✅ Preprocessing Complete. Final Shape: (1549, 28)
✅ Features Used: 19

🚀 Training RATIO PREDICTION Model
📊 Training on 1549 visits across 176 patients
🎯 Target: FVC_Ratio = Current_FVC / Baseline_FVC

FOLD 1/5

📈 FOLD 1 RESULTS:
  R²:   0.9355
  RMSE: 199.69 mL
  MAE:  130.93 mL
  LLL:  -6.5190

FOLD 2/5

📈 FOLD 2 RESULTS:
  R²:   0.9414
  RMSE: 215.84 mL
  MAE:  151.09 mL
  LLL:  -6.6674

FOLD 3/5

📈 FOLD 3 RESULTS:
  R²:   0.9052
  RMSE: 231.61 mL
  MAE:  159.39 mL
  LLL:  -6.7766

FOLD 4/5

📈 FOLD 4 RESULTS:
  R²:   0.9227
  RMSE: 212.80 mL
  MAE:  147.48 mL
  LLL:  -6.6250

FOLD 5/5

📈 FOLD 5 RESULTS:
  R²:   0.9511
  RMSE: 200.52 mL
  MAE:  132.51 mL
  LLL:  -6.5309

🏁 BASELINE RATIO PREDICTION RESULTS
R²:   0.9349   ✅ (Target > 0.88)
RMSE: 212.34 mL ❌ (Target < 170)
MAE:  144.19 mL
LLL:  -6.6231   ✅ (Target > -6.64)

🧪 OPTIONAL: SIGMA CALIBRATION
  σ×0.95 → LLL: -6.6207 (RMSE: 212.34) ✅
  σ×0.90